In [ ]:
"""Output Transactions

This notebook shows how LUSID uses synthetic transactions to fill in the gaps between user-instructed transactions and corporate actions.

Attributes
----------
instruments
transactions
portfolios
set holdings
output transactions
corporate actions
stock split
"""

# Output Transactions

This notebook shows how LUSID uses synthetic transactions to fill in the gaps between user-instructed transactions and corporate actions.


## Setup LUSID




In [2]:
# Import general purpose packages
import json
import os
import pandas as pd
import pprint
import printer as prettyprint
from datetime import datetime

# Import lusid specific packages
# These are the core lusid packages for interacting with the API via Python
import lusid
import lusid.models as models
from lusid.utilities import ApiClientFactory
from lusidjam.refreshing_token import RefreshingToken
from lusidtools.cocoon.cocoon import load_from_data_frame
from lusidtools.cocoon.cocoon_printer import (
    format_instruments_response,
    format_portfolios_response,
    format_transactions_response,
    format_quotes_response,
)
from lusidtools.pandas_utils.lusid_pandas import lusid_response_to_data_frame
from lusidtools.cocoon.utilities import create_scope_id

# Set pandas options
pd.set_option('display.max_columns', None)

# Authenticate our user and create our API client
secrets_path = os.getenv("FBN_SECRETS_PATH")

# Initiate an API Factory which is the client side object for interacting with LUSID APIs
api_factory = lusid.utilities.ApiClientFactory(
    token=RefreshingToken(),
    api_secrets_filename = secrets_path,
    app_name="LusidJupyterNotebook")

## Load source transactions


In [3]:
# Create a scope and portfolio code
scope = create_scope_id()
portfolio_code = "EQUITY_UK"

In [4]:
df = pd.read_csv("data/equity_transactions_for_output.csv")
df

,portfolio_code,portfolio_name,portfolio_base_currency,ticker,sedol,instrument_type,instrument_id,name,txn_id,txn_type,txn_trade_date,txn_settle_date,txn_units,txn_price,txn_consideration,currency,strategy,cash_transactions
0,EQUITY_UK,LUSID's top 10 FTSE stock portfolio,GBP,GB0002162385,SEDOL1,equity,EQ_1234,Aviva,trd_0001,StockIn,02/01/2020,04/01/2020,120000,5,600000,GBP,ftse_tracker,NaN
1,EQUITY_UK,LUSID's top 10 FTSE stock portfolio,GBP,GB0002162385,SEDOL1,equity,EQ_1234,Aviva,trd_0002,StockIn,02/01/2020,04/01/2020,12000,5,60000,GBP,ftse_tracker,NaN
2,EQUITY_UK,LUSID's top 10 FTSE stock portfolio,GBP,GB0000566504,SEDOL2,equity,EQ_1235,BHP,trd_0003,StockIn,02/01/2020,04/01/2020,60000,18,1080000,GBP,ftse_tracker,NaN
3,EQUITY_UK,LUSID's top 10 FTSE stock portfolio,GBP,GB0000566504,SEDOL2,equity,EQ_1235,BHP,trd_0004,StockIn,02/01/2020,04/01/2020,60000,18,1080000,GBP,ftse_tracker,NaN
4,EQUITY_UK,LUSID's top 10 FTSE stock portfolio,GBP,GB0031348658,SEDOL3,equity,EQ_1236,Barclays,trd_0005,StockIn,02/01/2020,04/01/2020,150000,2,300000,GBP,ftse_tracker,NaN
5,EQUITY_UK,LUSID's top 10 FTSE stock portfolio,GBP,GB0031348658,SEDOL3,equity,EQ_1236,Barclays,trd_0006,StockIn,02/01/2020,04/01/2020,150000,2,300000,GBP,ftse_tracker,NaN
6,EQUITY_UK,LUSID's top 10 FTSE stock portfolio,GBP,GB0007980591,SEDOL4,equity,EQ_1237,BP,trd_0007,StockIn,02/01/2020,04/01/2020,100000,5,500000,GBP,ftse_tracker,NaN
7,EQUITY_UK,LUSID's top 10 FTSE stock portfolio,GBP,GB0007980591,SEDOL4,equity,EQ_1237,BP,trd_0008,StockIn,02/01/2020,04/01/2020,100000,5,500000,GBP,ftse_tracker,NaN
8,EQUITY_UK,LUSID's top 10 FTSE stock portfolio,GBP,GB0005405286,SEDOL5,equity,EQ_1238,HSBC,trd_0009,StockIn,02/01/2020,04/01/2020,20000,6,120000,GBP,ftse_tracker,NaN
9,EQUITY_UK,LUSID's top 10 FTSE stock portfolio,GBP,GB0005405286,SEDOL5,equity,EQ_1238,HSBC,trd_0010,StockIn,02/01/2020,04/01/2020,20000,6,120000,GBP,ftse_tracker,NaN


## Load instruments

Create the instruments and add on ISIN and Sedol as identifiers

EXCLUDE CASH - completely until the dividend...

In [5]:
instrument_mapping = {
    "identifier_mapping": {
        "ClientInternal": "instrument_id",
        "Isin": "ticker",
        "Sedol": "sedol",
    },
    "required": {
        "name": "name"
    },
}

In [6]:
result = load_from_data_frame(
    api_factory=api_factory,
    scope=scope,
    data_frame=df,
    mapping_required=instrument_mapping["required"],
    mapping_optional={},
    file_type="instruments",
    identifier_mapping=instrument_mapping["identifier_mapping"],
)

succ, failed, errors = format_instruments_response(result)
pd.DataFrame(data=[{"success": len(succ), "failed": len(failed), "errors": len(errors)}])

,success,failed,errors
0,10,0,0


## Create portfolio

In [7]:
portfolio_mapping = {
    "required": {
        "code": "portfolio_code",
        "display_name": "portfolio_name",
        "base_currency": "$GBP",
    },
    "optional": {"created": "$2020-01-01T00:00:00+00:00"},
}

In [8]:
result = load_from_data_frame(
    api_factory=api_factory,
    scope=scope,
    data_frame=df,
    mapping_required=portfolio_mapping["required"],
    mapping_optional=portfolio_mapping["optional"],
    file_type="portfolios",
    sub_holding_keys=[],
)

succ, failed = format_portfolios_response(result)
pd.DataFrame(data=[{"success": len(succ), "failed": len(failed), "errors": len(errors)}])


,success,failed,errors
0,1,0,0


## Load in transactions

In [9]:
transaction_mapping = {
    "identifier_mapping": {
        "ClientInternal": "instrument_id",
    },
    "required": {
        "code": "portfolio_code",
        "transaction_id": "txn_id",
        "type": "txn_type",
        "transaction_price.price": "txn_price",
        "transaction_price.type": "$Price",
        "total_consideration.amount": "txn_consideration",
        "units": "txn_units",
        "transaction_date": "txn_trade_date",
        "total_consideration.currency": "portfolio_base_currency",
        "settlement_date": "txn_settle_date",        
    },
    "optional": {},
    "properties": [],
}

In [10]:
result = load_from_data_frame(
    api_factory=api_factory,
    scope=scope,
    data_frame=df,
    mapping_required=transaction_mapping["required"],
    mapping_optional=transaction_mapping["optional"],
    file_type="transactions",
    identifier_mapping=transaction_mapping["identifier_mapping"],
    property_columns=transaction_mapping["properties"],
    properties_scope=scope,
)

succ, failed = format_transactions_response(result)
pd.DataFrame(data=[{"success": len(succ), "failed": len(failed), "errors": len(failed)}])

,success,failed,errors
0,1,0,0


## Check holdings

In [11]:
# Define the transaction portfolio API
txn_portfolio_api = api_factory.build(lusid.api.TransactionPortfoliosApi)

response = txn_portfolio_api.get_holdings(
    scope=scope,
    code=portfolio_code,
    property_keys=["Instrument/default/Name"],
)

In [12]:
# Load a mapping file for DataFrame headers for the get holdings response
with open(r"config/get_holdings_mapping.json") as mappings_file:
    get_holdings_json_mapping = json.load(mappings_file)

holdings_df = lusid_response_to_data_frame(
    response,
    rename_properties=True,
    column_name_mapping=get_holdings_json_mapping,
)

holdings_df

,LusidInstrumentId,SubHoldingKeys,InstrumentName,SourcePortfolioId,SourcePortfolioScope(default-Properties),HoldingType,Units,SettledUnits,Amount-Cost,Currency-Cost,Amount-CostPortfolioCcy,Currenct-CostPortfolioCcy
0,LUID_KR3A1NMI,{},Aviva,EQUITY_UK,38a9-a5d2-97f2-9a,P,132000.0,132000.0,660000.0,GBP,0.0,GBP
1,LUID_WSHJKJ2Y,{},BHP,EQUITY_UK,38a9-a5d2-97f2-9a,P,120000.0,120000.0,2160000.0,GBP,0.0,GBP
2,LUID_SIMWQCNR,{},Barclays,EQUITY_UK,38a9-a5d2-97f2-9a,P,300000.0,300000.0,600000.0,GBP,0.0,GBP
3,LUID_80DILFAS,{},BP,EQUITY_UK,38a9-a5d2-97f2-9a,P,200000.0,200000.0,1000000.0,GBP,0.0,GBP
4,LUID_S1MNV9OQ,{},HSBC,EQUITY_UK,38a9-a5d2-97f2-9a,P,40000.0,40000.0,240000.0,GBP,0.0,GBP
5,LUID_49KIZM5K,{},Morrisons,EQUITY_UK,38a9-a5d2-97f2-9a,P,360000.0,360000.0,720000.0,GBP,0.0,GBP
6,LUID_AU5UQIVK,{},Tesco,EQUITY_UK,38a9-a5d2-97f2-9a,P,12000.0,12000.0,100000.0,GBP,0.0,GBP
7,LUID_00IPL9KJ,{},Rightmove,EQUITY_UK,38a9-a5d2-97f2-9a,P,160000.0,160000.0,960000.0,GBP,0.0,GBP
8,LUID_4C90VUEA,{},vodafone,EQUITY_UK,38a9-a5d2-97f2-9a,P,900000.0,900000.0,900000.0,GBP,0.0,GBP
9,LUID_Y92ZIAH5,{},Anglo American plc,EQUITY_UK,38a9-a5d2-97f2-9a,P,70000.0,70000.0,1400000.0,GBP,0.0,GBP


## Create corporate actions source

The corporate actions source is a container for holding corporate transactions. In this section we:
- Create a new corporate actions source
- Assign corporate actions source to our portfolio

In [13]:
ca_source_code = "ca_vendor1"

# Create first corporate action source

try:
    source_request = models.CreateCorporateActionSourceRequest(
        scope=scope,
        code=ca_source_code,
        display_name=ca_source_code,
        description="Corporate Actions source for Output Transactions notebook",
    )
    
    source_result = api_factory.build(
        lusid.api.CorporateActionSourcesApi
    ).create_corporate_action_source(create_corporate_action_source_request=source_request)
    
except:
    pass

In [14]:
# Assign corporate actions source to our portfolio

api_factory.build(lusid.api.TransactionPortfoliosApi).upsert_portfolio_details(
    scope=scope,
    code=portfolio_code,
    effective_at="2020-01-01T00:00:00+00:00",
    create_portfolio_details=lusid.models.CreatePortfolioDetails(
      corporate_action_source_id=lusid.ResourceId(
          scope=scope,
          code=ca_source_code,
      ),  
    ),
)

print(
    f"Corporate actions source of {ca_source_code} assigned to portfolio {portfolio_code}"
)

Corporate actions source of ca_vendor1 assigned to portfolio EQUITY_UK


## Load transitions from an external source

Transitions determine which Instrument is taking part in a corporate action, and what the effect of the corporate action is on holdings in that instrument.

For the transitions in our current example, we have:

* A bonus issue of 13,200 units from Aviva (1 bonus share per 10 held)
* A cash dividend of £204,000 from BHP (£1.70 GBP per share held)
* A 1:2 stock split from Tesco (each 1 share held is split into 2 shares held)

These transitions are posted with a payment date of 31 March 2020.

In [15]:
# Load transitions from a CSV file

corporate_action_df = pd.read_csv("data/corp-acts/corpact_transitions_split.csv")

# Format the datetimes into ISO strings
for col in ["announcement_date", "ex_date", "payment_date", "record_date"]:
    corporate_action_df[col] = corporate_action_df[col].apply(
        lambda x: datetime.strptime(x, "%d/%m/%Y").strftime(format="%Y-%m-%dT00:00:00Z")
    )

corporate_action_df

,code,action_description,description,announcement_date,ex_date,record_date,payment_date,client_id,input_instrument_name,input_units_factor,input_cost_factor,output_instrument_internal,output_units_factor,output_cost_factor,dividend_yield
0,5943592342,dividend-cash,BHP,2020-03-31T00:00:00Z,2020-03-31T00:00:00Z,2020-03-31T00:00:00Z,2020-03-31T00:00:00Z,EQ_1235,BHP,1,1,CCY_GBP,1.7,1,1.7
1,5943592343,bonus-issue,Aviva,2020-03-31T00:00:00Z,2020-03-31T00:00:00Z,2020-03-31T00:00:00Z,2020-03-31T00:00:00Z,EQ_1234,Aviva,1,1,EQ_1234,1.1,1,0.0
2,5943592344,stock-split,Tesco,2020-03-31T00:00:00Z,2020-03-31T00:00:00Z,2020-03-31T00:00:00Z,2020-03-31T00:00:00Z,EQ_1240,Tesco,1,1,EQ_1240,2.0,1,0.0


In [16]:
# Add environment specific output LUIDs to the dataset

out_instrument_luid = []

for index, item in corporate_action_df.iterrows():
    if item["output_instrument_internal"].startswith("CCY_"):\
        out_instrument_luid.append("nan")
    else:
        out_instrument_luid.append(
            api_factory.build(lusid.api.InstrumentsApi)
            .get_instrument(
                identifier_type="ClientInternal",
                identifier=item["output_instrument_internal"],
            )
            .lusid_instrument_id
        )
        
corporate_action_df["output_instrument_luid"] = out_instrument_luid

In [17]:
# Add environment specific input LUIDs to the dataset
# Change the identifier_type as required - this can be any unique identifier which LUSID can resolve to a LUID

corporate_action_df["input_instrument_luid"] = corporate_action_df["client_id"].apply(
    lambda x: api_factory.build(lusid.api.InstrumentsApi)
    .get_instrument(identifier_type="ClientInternal", identifier=x)
    .lusid_instrument_id
)

## Translate the DataFrame into  LUSID Transition

Here is the transition which needs to be translated into a LUSID Transition

In [18]:
corporate_action_df

,code,action_description,description,announcement_date,ex_date,record_date,payment_date,client_id,input_instrument_name,input_units_factor,input_cost_factor,output_instrument_internal,output_units_factor,output_cost_factor,dividend_yield,output_instrument_luid,input_instrument_luid
0,5943592342,dividend-cash,BHP,2020-03-31T00:00:00Z,2020-03-31T00:00:00Z,2020-03-31T00:00:00Z,2020-03-31T00:00:00Z,EQ_1235,BHP,1,1,CCY_GBP,1.7,1,1.7,nan,LUID_WSHJKJ2Y
1,5943592343,bonus-issue,Aviva,2020-03-31T00:00:00Z,2020-03-31T00:00:00Z,2020-03-31T00:00:00Z,2020-03-31T00:00:00Z,EQ_1234,Aviva,1,1,EQ_1234,1.1,1,0.0,LUID_KR3A1NMI,LUID_KR3A1NMI
2,5943592344,stock-split,Tesco,2020-03-31T00:00:00Z,2020-03-31T00:00:00Z,2020-03-31T00:00:00Z,2020-03-31T00:00:00Z,EQ_1240,Tesco,1,1,EQ_1240,2.0,1,0.0,LUID_AU5UQIVK,LUID_AU5UQIVK


In [19]:
# This code produces a dictionary of transitions where:
# The dict key is the corporate action code and the dict value is a list
# of input and output CorporateActionTransitionComponentRequest objects

transitions = {}
LUSID_INSTRUMENT_IDENTIFIER = "Instrument/default/LusidInstrumentId"

for index, ca in corporate_action_df.iterrows():
    
    # Create transition components
    cat_in = models.CorporateActionTransitionComponentRequest(
        instrument_identifiers={
            LUSID_INSTRUMENT_IDENTIFIER: ca["input_instrument_luid"]
        },
        units_factor=ca["input_units_factor"],
        cost_factor=ca["input_cost_factor"],
    )
    
    # Determine if the output is Cash or Stock, and create the 
    # appropriate transition component
    if str(ca["output_instrument_luid"]) == "nan":
        cat_out = models.CorporateActionTransitionComponentRequest(
            instrument_identifiers={
                "Instrument/default/Currency": ca["output_instrument_internal"][4:]
            },
            units_factor=ca["output_units_factor"],
            cost_factor=ca["output_cost_factor"],
        )
    else:
        cat_out = models.CorporateActionTransitionComponentRequest(
            instrument_identifiers={
                LUSID_INSTRUMENT_IDENTIFIER: ca["input_instrument_luid"]
            },
            units_factor=ca["output_units_factor"],
            cost_factor=ca["output_cost_factor"],
        )
        
    key = ca["code"]
    transitions.setdefault(key, [])
    transitions[key].append(ca["action_description"])
    transitions[key].append(cat_in)
    transitions[key].append(cat_out)

Use the transition to create an [UpsertCorporateActionRequest](https://www.lusid.com/docs/api/#operation/BatchUpsertCorporateActions)

In [20]:
# Iterate through the transitions, turining them into LUSID Corporate Action Requests

for key, values in transitions.items():
    transition_code = key
    transition_type = values[0]
    transition_in = values[1]
    transitions_out = values[2:]
    
    for x in transitions_out:
        if isinstance(x, str):
            transitions_out.remove(x)
            
    temp_transition = models.CorporateActionTransition(
        input_transition=transition_in, output_transitions=transitions_out
    )
    
    # Extract the data for the corporate action from the LUID corporate actions dataframe
    data = corporate_action_df[corporate_action_df["code"] == transition_code]
    
    # Iterate through each row of data and create the Corporate Action
    # Request for the appropriate type of action.
    for row, item in data.iterrows():
        if transition_type == "dividend-cash":
            div_ca = models.UpsertCorporateActionRequest(
                corporate_action_code=str(item["code"]),
                announcement_date=item["announcement_date"],
                ex_date=item["ex_date"],
                record_date=item["record_date"],
                payment_date=item["payment_date"],
                transitions=[temp_transition],
            )
        if transition_type == "bonus-issue":
            bonus_ca = models.UpsertCorporateActionRequest(
                corporate_action_code=str(item["code"]),
                announcement_date=item["announcement_date"],
                ex_date=item["ex_date"],
                record_date=item["record_date"],
                payment_date=item["payment_date"],
                transitions=[temp_transition],
            )
        if transition_type == "stock-split":
            split_ca = models.UpsertCorporateActionRequest(
                corporate_action_code=str(item["code"]),
                announcement_date=item["announcement_date"],
                ex_date=item["ex_date"],
                record_date=item["record_date"],
                payment_date=item["payment_date"],
                transitions=[temp_transition],
            )
        
            

## Upsert the corporate actions into LUSID's movements engine

In [21]:
result = api_factory.build(
    lusid.api.CorporateActionSourcesApi
).batch_upsert_corporate_actions(
    scope=scope, 
    code=ca_source_code, 
    upsert_corporate_action_request=[div_ca, bonus_ca, split_ca],
)

## Check holdings post-CA

The portfolio should now have:
* 145,200 units of Aviva (+ 13,200 units)
* £204,000 GBP cash (+ £204,000 cash)
* 24,000 units of Tesco (+12,000 units)

In [22]:
response = txn_portfolio_api.get_holdings(
    scope=scope,
    code=portfolio_code,
    property_keys=["Instrument/default/Name"],
)

holdings_df = lusid_response_to_data_frame(
    response,
    rename_properties=True,
    column_name_mapping=get_holdings_json_mapping,
)

holdings_df

,LusidInstrumentId,SubHoldingKeys,InstrumentName,SourcePortfolioId,SourcePortfolioScope(default-Properties),HoldingType,Units,SettledUnits,Amount-Cost,Currency-Cost,Amount-CostPortfolioCcy,Currenct-CostPortfolioCcy
0,LUID_KR3A1NMI,{},Aviva,EQUITY_UK,38a9-a5d2-97f2-9a,P,145200.0,145200.0,660000.0,GBP,0.0,GBP
1,LUID_WSHJKJ2Y,{},BHP,EQUITY_UK,38a9-a5d2-97f2-9a,P,120000.0,120000.0,2160000.0,GBP,0.0,GBP
2,LUID_SIMWQCNR,{},Barclays,EQUITY_UK,38a9-a5d2-97f2-9a,P,300000.0,300000.0,600000.0,GBP,0.0,GBP
3,LUID_80DILFAS,{},BP,EQUITY_UK,38a9-a5d2-97f2-9a,P,200000.0,200000.0,1000000.0,GBP,0.0,GBP
4,LUID_S1MNV9OQ,{},HSBC,EQUITY_UK,38a9-a5d2-97f2-9a,P,40000.0,40000.0,240000.0,GBP,0.0,GBP
5,LUID_49KIZM5K,{},Morrisons,EQUITY_UK,38a9-a5d2-97f2-9a,P,360000.0,360000.0,720000.0,GBP,0.0,GBP
6,LUID_AU5UQIVK,{},Tesco,EQUITY_UK,38a9-a5d2-97f2-9a,P,24000.0,24000.0,100000.0,GBP,0.0,GBP
7,LUID_00IPL9KJ,{},Rightmove,EQUITY_UK,38a9-a5d2-97f2-9a,P,160000.0,160000.0,960000.0,GBP,0.0,GBP
8,LUID_4C90VUEA,{},vodafone,EQUITY_UK,38a9-a5d2-97f2-9a,P,900000.0,900000.0,900000.0,GBP,0.0,GBP
9,LUID_Y92ZIAH5,{},Anglo American plc,EQUITY_UK,38a9-a5d2-97f2-9a,P,70000.0,70000.0,1400000.0,GBP,0.0,GBP


## Adjust holdings

example text... 
(Maybe we say that there actually should have been a cash balance in there that was not part of the transactions... manually adjust the holdings)
This allows us to set the holdings of a portfolio to a given state. In this case we will add in a starting cash balance. Let us give the analyst a starting cash balance of £30,000,000. We will make this balance effective from just after the creation of our portfolio.

In [23]:
# Set the date from which the cash balance will apply to be the start of the portfolio
response = txn_portfolio_api.get_details(
    scope=scope,
    code=portfolio_code,
)

holdings_effective_date = response.version.effective_from

# Define the actual initial cash balance
initial_cash_balance = 500000

# Create a holding adjustment to set our initial cash balance
holding_adjustment = [
    models.AdjustHoldingRequest(
        instrument_identifiers={
            "Instrument/default/Currency": "GBP"},
        tax_lots=[
            models.TargetTaxLotRequest(
                units=initial_cash_balance,
            )
        ]
    )
]

# Call LUSID to set our initial cash balance
set_holdings_response = api_factory.build(lusid.api.TransactionPortfoliosApi).set_holdings(
    scope=scope,
    code=portfolio_code,
    effective_at=holdings_effective_date,
    adjust_holding_request=holding_adjustment,
)

## Check holdings post-Adjustment

The portfolio should now have:
* £704,000 GBP cash (+ £500,000 cash)


In [24]:
response = txn_portfolio_api.get_holdings(
    scope=scope,
    code=portfolio_code,
    property_keys=["Instrument/default/Name"],
)

holdings_df = lusid_response_to_data_frame(
    response,
    rename_properties=True,
    column_name_mapping=get_holdings_json_mapping,
)

holdings_df

,LusidInstrumentId,SubHoldingKeys,InstrumentName,SourcePortfolioId,SourcePortfolioScope(default-Properties),HoldingType,Units,SettledUnits,Amount-Cost,Currency-Cost,Amount-CostPortfolioCcy,Currenct-CostPortfolioCcy
0,LUID_KR3A1NMI,{},Aviva,EQUITY_UK,38a9-a5d2-97f2-9a,P,145200.0,145200.0,660000.0,GBP,0.0,GBP
1,LUID_WSHJKJ2Y,{},BHP,EQUITY_UK,38a9-a5d2-97f2-9a,P,120000.0,120000.0,2160000.0,GBP,0.0,GBP
2,LUID_SIMWQCNR,{},Barclays,EQUITY_UK,38a9-a5d2-97f2-9a,P,300000.0,300000.0,600000.0,GBP,0.0,GBP
3,LUID_80DILFAS,{},BP,EQUITY_UK,38a9-a5d2-97f2-9a,P,200000.0,200000.0,1000000.0,GBP,0.0,GBP
4,LUID_S1MNV9OQ,{},HSBC,EQUITY_UK,38a9-a5d2-97f2-9a,P,40000.0,40000.0,240000.0,GBP,0.0,GBP
5,LUID_49KIZM5K,{},Morrisons,EQUITY_UK,38a9-a5d2-97f2-9a,P,360000.0,360000.0,720000.0,GBP,0.0,GBP
6,LUID_AU5UQIVK,{},Tesco,EQUITY_UK,38a9-a5d2-97f2-9a,P,24000.0,24000.0,100000.0,GBP,0.0,GBP
7,LUID_00IPL9KJ,{},Rightmove,EQUITY_UK,38a9-a5d2-97f2-9a,P,160000.0,160000.0,960000.0,GBP,0.0,GBP
8,LUID_4C90VUEA,{},vodafone,EQUITY_UK,38a9-a5d2-97f2-9a,P,900000.0,900000.0,900000.0,GBP,0.0,GBP
9,LUID_Y92ZIAH5,{},Anglo American plc,EQUITY_UK,38a9-a5d2-97f2-9a,P,70000.0,70000.0,1400000.0,GBP,0.0,GBP


## What transactions make up our holdings?

In [25]:
# Load a mapping file for DataFrame headers for the build transaction response
with open(r"config/build_transactions_mapping.json") as mappings_file:
    build_transactions_json_mapping = json.load(mappings_file)


build_transactions_response = txn_portfolio_api.build_transactions(
    scope=scope,
    code=portfolio_code,
    transaction_query_parameters=models.TransactionQueryParameters(
        start_date="2020-01-01", end_date="2020-12-31"
    ),
    property_keys=["Instrument/default/Name"],
)

build_transactions_df = lusid_response_to_data_frame(
    build_transactions_response,
    rename_properties=True,
    column_name_mapping=build_transactions_json_mapping,
)
build_transactions_df

,TransactionId,TransactionType,TransactionTypeDesc,ClientId,LusidInstrumentId,TransactionDate,SettlementDate,Units,TransactionAmount,Price,PriceType,TotalConsideration,TotalConsiderationCurrency,ExchangeRate,TransactionToPortfolioRate,TransactionCurrency,SourcePortfolioId(default-Properties),SourcePortfolioScope(default-Properties),ResultantHolding,InstrumentName,source,TransactionStatus,EntryDateTime,RealisedGainLoss,instrument_identifiers.Instrument/default/LusidInstrumentId,OriginalTradeDate(default-Properties),OriginalSettlementDate(default-Properties),InstrumentCurrency,TradeToPortfolioRate(default-Properties)
0,trd_0001,StockIn,Transfer In,EQ_1234,LUID_KR3A1NMI,2020-01-02 00:00:00+00:00,2020-01-04 00:00:00+00:00,120000.0,600000.0,5.0,Price,600000.0,GBP,1.0,0.0,GBP,EQUITY_UK,38a9-a5d2-97f2-9a,120000.0,Aviva,None,Active,2020-07-16 15:20:42.918073+00:00,[],NaN,NaN,NaN,NaN,NaN
1,trd_0002,StockIn,Transfer In,EQ_1234,LUID_KR3A1NMI,2020-01-02 00:00:00+00:00,2020-01-04 00:00:00+00:00,12000.0,60000.0,5.0,Price,60000.0,GBP,1.0,0.0,GBP,EQUITY_UK,38a9-a5d2-97f2-9a,132000.0,Aviva,None,Active,2020-07-16 15:20:42.918073+00:00,[],NaN,NaN,NaN,NaN,NaN
2,trd_0003,StockIn,Transfer In,EQ_1235,LUID_WSHJKJ2Y,2020-01-02 00:00:00+00:00,2020-01-04 00:00:00+00:00,60000.0,1080000.0,18.0,Price,1080000.0,GBP,1.0,0.0,GBP,EQUITY_UK,38a9-a5d2-97f2-9a,60000.0,BHP,None,Active,2020-07-16 15:20:42.918073+00:00,[],NaN,NaN,NaN,NaN,NaN
3,trd_0004,StockIn,Transfer In,EQ_1235,LUID_WSHJKJ2Y,2020-01-02 00:00:00+00:00,2020-01-04 00:00:00+00:00,60000.0,1080000.0,18.0,Price,1080000.0,GBP,1.0,0.0,GBP,EQUITY_UK,38a9-a5d2-97f2-9a,120000.0,BHP,None,Active,2020-07-16 15:20:42.918073+00:00,[],NaN,NaN,NaN,NaN,NaN
4,trd_0005,StockIn,Transfer In,EQ_1236,LUID_SIMWQCNR,2020-01-02 00:00:00+00:00,2020-01-04 00:00:00+00:00,150000.0,300000.0,2.0,Price,300000.0,GBP,1.0,0.0,GBP,EQUITY_UK,38a9-a5d2-97f2-9a,150000.0,Barclays,None,Active,2020-07-16 15:20:42.918073+00:00,[],NaN,NaN,NaN,NaN,NaN
5,trd_0006,StockIn,Transfer In,EQ_1236,LUID_SIMWQCNR,2020-01-02 00:00:00+00:00,2020-01-04 00:00:00+00:00,150000.0,300000.0,2.0,Price,300000.0,GBP,1.0,0.0,GBP,EQUITY_UK,38a9-a5d2-97f2-9a,300000.0,Barclays,None,Active,2020-07-16 15:20:42.918073+00:00,[],NaN,NaN,NaN,NaN,NaN
6,trd_0007,StockIn,Transfer In,EQ_1237,LUID_80DILFAS,2020-01-02 00:00:00+00:00,2020-01-04 00:00:00+00:00,100000.0,500000.0,5.0,Price,500000.0,GBP,1.0,0.0,GBP,EQUITY_UK,38a9-a5d2-97f2-9a,100000.0,BP,None,Active,2020-07-16 15:20:42.918073+00:00,[],NaN,NaN,NaN,NaN,NaN
7,trd_0008,StockIn,Transfer In,EQ_1237,LUID_80DILFAS,2020-01-02 00:00:00+00:00,2020-01-04 00:00:00+00:00,100000.0,500000.0,5.0,Price,500000.0,GBP,1.0,0.0,GBP,EQUITY_UK,38a9-a5d2-97f2-9a,200000.0,BP,None,Active,2020-07-16 15:20:42.918073+00:00,[],NaN,NaN,NaN,NaN,NaN
8,trd_0009,StockIn,Transfer In,EQ_1238,LUID_S1MNV9OQ,2020-01-02 00:00:00+00:00,2020-01-04 00:00:00+00:00,20000.0,120000.0,6.0,Price,120000.0,GBP,1.0,0.0,GBP,EQUITY_UK,38a9-a5d2-97f2-9a,20000.0,HSBC,None,Active,2020-07-16 15:20:42.918073+00:00,[],NaN,NaN,NaN,NaN,NaN
9,trd_0010,StockIn,Transfer In,EQ_1238,LUID_S1MNV9OQ,2020-01-02 00:00:00+00:00,2020-01-04 00:00:00+00:00,20000.0,120000.0,6.0,Price,120000.0,GBP,1.0,0.0,GBP,EQUITY_UK,38a9-a5d2-97f2-9a,40000.0,HSBC,None,Active,2020-07-16 15:20:42.918073+00:00,[],NaN,NaN,NaN,NaN,NaN
